In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots


from sqlalchemy import create_engine


In [2]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'
TABLE_SCHEMA = 'diploma'
ENGINE = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')

query_users = f'''SELECT * FROM {TABLE_SCHEMA}.users'''
query_geo_daily_report = f'''SELECT * FROM {TABLE_SCHEMA}.geo_daily_report'''

# Рассчитаем ltv по формуле (LTV = revenue(iap + ad) / кол-во игроков) за период  с '2022-04-01' по '2022-05-01' таблица.geo_daily_report исключая органику

Рассчитаем общий ltv по формуле (LTV = revenue(iap + ad) / кол-во игроков) за период  с '2022-04-01' по '2022-05-01' без учета органики

In [6]:
# Рассчитаем общий ltv по формуле (LTV = revenue(iap + ad) / кол-во игроков)
# за период  с '2022-04-01' по '2022-05-01' 

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

ltv_sql = pd.read_sql(f'''
SELECT
    (SUM(af_ad_revenue_usd) + SUM(af_purchase_usd)) / SUM (installs) AS ltv_iap
FROM {TABLE_SCHEMA}.geo_daily_report 
WHERE
    1=1
    AND project_id = '{data_project_id}' 
    AND event_date >='{data_start}'
    AND event_date < '{data_end}'
    AND media_source != 'Organic' --исключая органику

''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
ltv_sql.head(2)

execution time: 0:00:00.192769


,ltv_iap
0,0.418292


In [9]:
# Рассчитаем общий ltv по формуле (LTV = revenue(iap + ad) / кол-во игроков) 
# за период  с '2022-04-01' по '2022-05-01'

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

ltv_sql = pd.read_sql(f'''
SELECT
    (SUM(revenue_usd) + SUM(ad_revenue_usd)) / COUNT (DISTINCT appsflyer_id) AS ltv_iap
FROM {TABLE_SCHEMA}.users 
WHERE
    1=1
    AND project_id = '{data_project_id}' 
    AND install_date >='{data_start}'
    AND install_date < '{data_end}'
    AND media_source != 'Organic' --исключая органику

''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
ltv_sql.head(2)

execution time: 0:00:00.577089


,ltv_iap
0,0.397701


Рассчитаем общий ltv по формуле (LTV = revenue(iap + ad) / кол-во игроков) за период  с '2022-04-01' по '2022-05-01' без учета органики по media_source

In [3]:
# Рассчитаем общий ltv по формуле (LTV = REVENUE / кол-во игроков) по media_source

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

ltv_media = pd.read_sql(f'''
WITH
-- разделяем когорты по типу трафика и по условию
    user_types AS (
        SELECT
            *,
            CASE WHEN media_source = 'bytedanceglobal_int' THEN af_ad_revenue_usd END AS af_ad_rev_bytedanceglobal_int,
            CASE WHEN media_source = 'googleadwords_int' THEN af_ad_revenue_usd END AS af_ad_rev_googleadwords_int,
            CASE WHEN media_source = 'Facebook Ads' THEN af_ad_revenue_usd END AS af_ad_rev_Facebook,
            
            CASE WHEN media_source = 'bytedanceglobal_int' THEN af_purchase_usd END AS af_purchase_usd_bytedanceglobal_int,
            CASE WHEN media_source = 'googleadwords_int' THEN af_purchase_usd END AS af_purchase_usd_googleadwords_int,
            CASE WHEN media_source = 'Facebook Ads' THEN af_purchase_usd END AS af_purchase_usd_Facebook,
            
            CASE WHEN media_source = 'bytedanceglobal_int' THEN installs END AS installs_bytedanceglobal_int,
            CASE WHEN media_source = 'googleadwords_int' THEN installs END AS installs_googleadwords_int,
            CASE WHEN media_source = 'Facebook Ads' THEN installs END AS installs_Facebook 
        FROM {TABLE_SCHEMA}.geo_daily_report
        WHERE
            1=1
            AND project_id = '{data_project_id}' 
            AND event_date >='{data_start}'
            AND event_date < '{data_end}'
            AND media_source != 'Organic' --исключая органику
)
SELECT
    --event_date::date,
    (SUM(af_ad_rev_bytedanceglobal_int)
    + SUM(af_purchase_usd_bytedanceglobal_int)) / SUM(installs_bytedanceglobal_int) as ltv__bytedanceglobal,
    (SUM(af_ad_rev_googleadwords_int)
    + SUM(af_purchase_usd_googleadwords_int)) / SUM(installs_googleadwords_int) as ltv_googleadwords,
    (SUM(af_ad_rev_Facebook)
    + SUM(af_purchase_usd_Facebook)) / SUM(installs_Facebook) as ltv_Facebook   
FROM 
    user_types 
--GROUP BY 1

''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
ltv_media.head(2)

execution time: 0:00:00.796332


,ltv__bytedanceglobal,ltv_googleadwords,ltv_facebook
0,0.398418,0.0,1.333321


Рассчитаем общий ltv по формуле (LTV = revenue(iap + ad) / кол-во игроков) за период  с '2022-04-01' по '2022-05-01' без учета органики по топ 5 странам

In [16]:
# Рассчитаем общий ltv по формуле (LTV = REVENUE / кол-во игроков) по топ 5 странам 

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

ltv_country = pd.read_sql(f'''
WITH
-- разделяем когорты по типу трафика и по условию
    user_types AS (
        SELECT
            *,
            CASE WHEN country_code = 'US' THEN af_ad_revenue_usd END AS af_ad_rev_us,
            CASE WHEN country_code = 'MY' THEN af_ad_revenue_usd END AS af_ad_rev_my,
            CASE WHEN country_code = 'PH' THEN af_ad_revenue_usd END AS af_ad_rev_ph,
            CASE WHEN country_code = 'UK' THEN af_ad_revenue_usd END AS af_ad_rev_uk,
            CASE WHEN country_code = 'ID' THEN af_ad_revenue_usd END AS af_ad_rev_id,
            
            CASE WHEN country_code = 'US' THEN af_purchase_usd END AS af_purchase_usd_us,
            CASE WHEN country_code = 'MY' THEN af_purchase_usd END AS af_purchase_usd_my,
            CASE WHEN country_code = 'PH' THEN af_purchase_usd END AS af_purchase_usd_ph,
            CASE WHEN country_code = 'UK' THEN af_purchase_usd END AS af_purchase_usd_uk,
            CASE WHEN country_code = 'ID' THEN af_purchase_usd END AS af_purchase_usd_id,
            
            CASE WHEN country_code = 'US' THEN installs END AS installs_us,
            CASE WHEN country_code = 'MY' THEN installs END AS installs_my,
            CASE WHEN country_code = 'PH' THEN installs END AS installs_ph,
            CASE WHEN country_code = 'UK' THEN installs END AS installs_uk, 
            CASE WHEN country_code = 'ID' THEN installs END AS installs_id 
        FROM {TABLE_SCHEMA}.geo_daily_report
        WHERE
            1=1
            AND project_id = '{data_project_id}' 
            AND event_date >='{data_start}'
            AND event_date < '{data_end}'
)
SELECT
    --event_date::date,
    (SUM(af_ad_rev_us) + SUM(af_purchase_usd_us)) / SUM(installs_us) as ltv_us,
    (SUM(af_ad_rev_my) + SUM(af_purchase_usd_my)) / SUM(installs_my) as ltv_my,
    (SUM(af_ad_rev_ph) + SUM(af_purchase_usd_ph)) / SUM(installs_ph) as ltv_ph, 
    (SUM(af_ad_rev_uk) + SUM(af_purchase_usd_uk)) / SUM(installs_uk) as ltv_uk,
    (SUM(af_ad_rev_id) + SUM(af_purchase_usd_id)) / SUM(installs_id) as ltv_id
FROM 
    user_types 
--GROUP BY 1

''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
ltv_country.head(2)

execution time: 0:00:00.190832


,ltv_us,ltv_my,ltv_ph,ltv_uk,ltv_id
0,0.70199,0.093373,0.075897,0.455212,0.14526


Рассчитаем накопительные ltv по формуле (LTV = revenue(iap + ad) / кол-во игроков) за период  с '2022-04-01' по '2022-05-01' без учета органики 

In [17]:
# Рассчитаем ltv по формуле (LTV = revenue(iap + ad) / кол-во игроков) за период  без учета органики по дням

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

ltv_sql = pd.read_sql(f'''
WITH ltv AS(
SELECT
    event_date::date,
    (SUM(af_ad_revenue_usd) + SUM(af_purchase_usd)) / SUM (installs) AS ltv_iap
FROM {TABLE_SCHEMA}.geo_daily_report 
WHERE
    1=1
    AND project_id = '{data_project_id}' 
    AND event_date >='{data_start}'
    AND event_date < '{data_end}'
    AND media_source != 'Organic' --исключая органику
GROUP BY 1
)
SELECT 
    event_date,
    ltv_iap,
    SUM(ltv_iap) OVER (ORDER BY event_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as ltv_iap_cumsum
   
FROM
    ltv
''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
ltv_sql.head(2)

execution time: 0:00:00.190073


,event_date,ltv_iap,ltv_iap_cumsum
0,2022-04-01,0.091895,0.091895
1,2022-04-02,0.162684,0.254578


In [18]:
# Дневной график накопительного ltv

x = ltv_sql.event_date              
y2 = ltv_sql.ltv_iap_cumsum


fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y2,  name='ltv_cumsum', mode='lines+markers',))

fig.update_layout(legend_orientation="v",
                  legend=dict(x=.5, xanchor="center"),
                  hovermode="x",
                  title="Дневной график накопительного ltv",
                  margin=dict(l=0, r=0, t=30, b=0))
fig.update_traces(hoverinfo="all", hovertemplate="event_date: %{x}<br>ltv_cumsum: %{y}")
fig.show()

In [19]:
# Средний LTV за период
ltv_sql.ltv_iap.mean()

0.49219514207930004

In [20]:
# Рассчитаем ltv по формуле (LTV = REVENUE / кол-во игроков) по media_source по дням

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

ltv_user_type = pd.read_sql(f'''
WITH
-- разделяем когорты по типу трафика и по условию
    user_types AS (
        SELECT
            *,
            CASE WHEN media_source = 'bytedanceglobal_int' THEN af_ad_revenue_usd END AS af_ad_rev_bytedanceglobal_int,
            CASE WHEN media_source = 'googleadwords_int' THEN af_ad_revenue_usd END AS af_ad_rev_googleadwords_int,
            CASE WHEN media_source = 'Facebook Ads' THEN af_ad_revenue_usd END AS af_ad_rev_Facebook,
            
            CASE WHEN media_source = 'bytedanceglobal_int' THEN af_purchase_usd END AS af_purchase_usd_bytedanceglobal_int,
            CASE WHEN media_source = 'googleadwords_int' THEN af_purchase_usd END AS af_purchase_usd_googleadwords_int,
            CASE WHEN media_source = 'Facebook Ads' THEN af_purchase_usd END AS af_purchase_usd_Facebook,
            
            CASE WHEN media_source = 'bytedanceglobal_int' THEN installs END AS installs_bytedanceglobal_int,
            CASE WHEN media_source = 'googleadwords_int' THEN installs END AS installs_googleadwords_int,
            CASE WHEN media_source = 'Facebook Ads' THEN installs END AS installs_Facebook 
        FROM {TABLE_SCHEMA}.geo_daily_report
        WHERE
            1=1
            AND project_id = '{data_project_id}' 
            AND event_date >='{data_start}'
            AND event_date < '{data_end}'
            AND media_source != 'Organic' --исключая органику
),
ltv AS (
SELECT
    event_date::date,
    (SUM(af_ad_rev_bytedanceglobal_int)
    + SUM(af_purchase_usd_bytedanceglobal_int)) / SUM(installs_bytedanceglobal_int) as ltv__bytedanceglobal,
    (SUM(af_ad_rev_googleadwords_int)
    + SUM(af_purchase_usd_googleadwords_int)) / SUM(installs_googleadwords_int) as ltv_googleadwords,
    (SUM(af_ad_rev_Facebook)
    + SUM(af_purchase_usd_Facebook)) / SUM(installs_Facebook) as ltv_Facebook  
FROM 
    user_types 
GROUP BY 1
)
SELECT 
    event_date,
    SUM(ltv__bytedanceglobal) OVER (ORDER BY event_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as ltv__bytedanceglobal_cumsum,
    SUM(ltv_googleadwords) OVER (ORDER BY event_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as ltv_googleadwords_cumsum,
    SUM(ltv_Facebook) OVER (ORDER BY event_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as ltv_Facebook_cumsum
    
FROM
    ltv
''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
ltv_user_type.head(2)

execution time: 0:00:00.189817


,event_date,ltv__bytedanceglobal_cumsum,ltv_googleadwords_cumsum,ltv_facebook_cumsum
0,2022-04-01,0.091895,NaN,NaN
1,2022-04-02,0.254796,NaN,0.0


In [21]:
# Дневной график накопительный ltv по каналам за период с '2022-04-01' по '2022-05-01' по дням

x = ltv_user_type.event_date                   
y2 = ltv_user_type.ltv__bytedanceglobal_cumsum
y3 = ltv_user_type.ltv_googleadwords_cumsum      
y4 = ltv_user_type.ltv_facebook_cumsum


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y2,  name='ltv__bytedanceglobal_cumsum', mode='lines+markers',))
fig.add_trace(go.Scatter(x=x, y=y3,  name='ltv_googleadwords_cumsum', mode='lines+markers',))
fig.add_trace(go.Scatter(x=x, y=y4,  name='ltv_facebook_cumsum', mode='lines+markers',))

fig.update_layout(legend_orientation="v",
                  legend=dict(x=.5, xanchor="center"),
                  hovermode="x",
                  title="Дневной накопительный график ltv по каналам",
                  margin=dict(l=0, r=0, t=30, b=0))
fig.update_traces(hoverinfo="all", hovertemplate="event_date: %{x}<br>ltv: %{y}")
fig.show()

In [22]:
# Рассчитаем ltv по формуле (LTV = REVENUE / кол-во игроков) по country_code за период  с '2022-04-01' по '2022-05-01'

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

ltv_country_code = pd.read_sql(f'''
WITH
-- разделяем когорты по типу трафика и по условию
    user_types AS (
        SELECT
            *,
            CASE WHEN country_code = 'US' THEN af_ad_revenue_usd END AS af_ad_rev_us,
            CASE WHEN country_code = 'MY' THEN af_ad_revenue_usd END AS af_ad_rev_my,
            CASE WHEN country_code = 'PH' THEN af_ad_revenue_usd END AS af_ad_rev_ph,
            CASE WHEN country_code = 'UK' THEN af_ad_revenue_usd END AS af_ad_rev_uk,
            CASE WHEN country_code = 'ID' THEN af_ad_revenue_usd END AS af_ad_rev_id,
            
            CASE WHEN country_code = 'US' THEN af_purchase_usd END AS af_purchase_usd_us,
            CASE WHEN country_code = 'MY' THEN af_purchase_usd END AS af_purchase_usd_my,
            CASE WHEN country_code = 'PH' THEN af_purchase_usd END AS af_purchase_usd_ph,
            CASE WHEN country_code = 'UK' THEN af_purchase_usd END AS af_purchase_usd_uk,
            CASE WHEN country_code = 'ID' THEN af_purchase_usd END AS af_purchase_usd_id,
            
            CASE WHEN country_code = 'US' THEN installs END AS installs_us,
            CASE WHEN country_code = 'MY' THEN installs END AS installs_my,
            CASE WHEN country_code = 'PH' THEN installs END AS installs_ph,
            CASE WHEN country_code = 'UK' THEN installs END AS installs_uk, 
            CASE WHEN country_code = 'ID' THEN installs END AS installs_id 
        FROM {TABLE_SCHEMA}.geo_daily_report
        WHERE
            1=1
            AND project_id = '{data_project_id}' 
            AND event_date >='{data_start}'
            AND event_date < '{data_end}'
),
ltv AS (
SELECT
    event_date::date,
    (SUM(af_ad_rev_us) + SUM(af_purchase_usd_us)) / SUM(installs_us) as ltv_us,
    (SUM(af_ad_rev_my) + SUM(af_purchase_usd_my)) / SUM(installs_my) as ltv_my,
    (SUM(af_ad_rev_ph) + SUM(af_purchase_usd_ph)) / SUM(installs_ph) as ltv_ph, 
    (SUM(af_ad_rev_uk) + SUM(af_purchase_usd_uk)) / SUM(installs_uk) as ltv_uk,
    (SUM(af_ad_rev_id) + SUM(af_purchase_usd_id)) / SUM(installs_id) as ltv_id
FROM 
    user_types 
GROUP BY 1
)
SELECT 
    event_date,
    SUM(ltv_us) OVER (ORDER BY event_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as ltv_us,
    SUM(ltv_my) OVER (ORDER BY event_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as ltv_my,
    SUM(ltv_ph) OVER (ORDER BY event_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as ltv_ph,
    SUM(ltv_uk) OVER (ORDER BY event_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as ltv_uk,
    SUM(ltv_id) OVER (ORDER BY event_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as ltv_id
FROM
    ltv
''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
ltv_country_code.head(2)

execution time: 0:00:00.198074


,event_date,ltv_us,ltv_my,ltv_ph,ltv_uk,ltv_id
0,2022-04-01,0.415724,0.043825,0.019660,0.071643,0.183119
1,2022-04-02,1.188330,0.092609,0.045639,0.071643,0.186417


In [23]:
# Дневной график накопительного ltv по по топ 5 стран за период с '2022-04-01' по '2022-05-01' по дням

x = ltv_country_code.event_date              
y1 = ltv_country_code.ltv_us      
y2 = ltv_country_code.ltv_my
y3 = ltv_country_code.ltv_ph      
y4 = ltv_country_code.ltv_uk
y5 = ltv_country_code.ltv_id

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y1,  name='ltv_us', mode='lines+markers',))
fig.add_trace(go.Scatter(x=x, y=y2,  name='ltv_my', mode='lines+markers',))
fig.add_trace(go.Scatter(x=x, y=y3,  name='ltv_ph', mode='lines+markers',))
fig.add_trace(go.Scatter(x=x, y=y4,  name='ltv_uk', mode='lines+markers',))
fig.add_trace(go.Scatter(x=x, y=y4,  name='ltv_id', mode='lines+markers',))

fig.update_layout(legend_orientation="v",
                  legend=dict(x=.5, xanchor="center"),
                  hovermode="x",
                  title="# Дневной графикнакопительного  ltv по по топ 5 стран",
                  margin=dict(l=0, r=0, t=30, b=0))
fig.update_traces(hoverinfo="all", hovertemplate="event_date: %{x}<br>ltv: %{y}")
fig.show()